In [9]:
import pandas as pd
import numpy as np

from keras import Input
from keras.engine import Model
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Concatenate, concatenate
from keras.callbacks import TensorBoard
from keras.models import Sequential

In [14]:
def build_model(features, data_length):
    model = Sequential()
    model.add(LSTM(64, input_shape=(data_length,1)))
    model.add(Dense(3, activation='softmax', name='IsSpike'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
#     inputs_list = [] 
#     for feature_name in features:
#         inputs_list.append((Input(shape=(data_length,1), name=feature_name)))
    
#     layers = [] 
#     for i, input_name in enumerate(inputs_list): 
#         layers.append(LSTM(64, return_sequences=False)(inputs_list[i]) )
        
#     #output = concatenate(layers) 
#     output = Dense(3, activation='softmax', name='IsSpike')(output)
    
#     model = Model(
#         inputs = inputs_list,
#         outputs = [output]
#     )
    
#     model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    return model    

data_length = 12

In [4]:
from sklearn.preprocessing import MinMaxScaler

master_df = pd.read_csv('C:/Users/Shoya/surf/data/master_df_v3.csv', encoding='latin1')
df = master_df[['Timestamp', 'Close', 'Volume_(BTC)', 'Volume_(Currency)', 'Date(UTC)', 'Bitcoin (Adj.Overlap)', 
               'Close Price % Change', 'Close Price % Change (Abs)', 'Is Spike']]

trends_df = pd.read_csv('C:/Users/Shoya/surf/data/Adj_Google_Trends_for_Mult_Keywords.csv', encoding='latin1') 
trends_cols = ["BitstampTrend", "CoinbaseTrend", "EthereumTrend", "BubbleTrend", "CryptocurrencyTrend", 
               "HashingTrend", "BlockchainTrend" ]

df[trends_cols] = trends_df[trends_cols]

display(df.head())

# lag inputs depending on data_length 
df['Price_lagged'] = df['Close']#.shift(data_length
df['Volume_BTC'] = df['Volume_(BTC)']#.shift(data_length)
df['Bitcoin_Adj'] = df['Bitcoin (Adj.Overlap)']#.shift(data_length)

df = df.dropna()
cols = ['Volume_BTC','Bitcoin_Adj', 'Price_lagged',"BitstampTrend", "CoinbaseTrend", "EthereumTrend", "BubbleTrend", "CryptocurrencyTrend", 
               "HashingTrend", "BlockchainTrend"]

# Stationalize Data by taking log differences
data_array = np.diff(np.log(df[cols]), axis=0)

df_2 = pd.DataFrame(data_array, columns=cols)
df_2 = df_2.replace([np.inf, -np.inf], np.nan).dropna()
display(df_2.head())

# writer = pd.ExcelWriter("check.xlsx",  engine='xlsxwriter', options={'remove_timezone': True})
# df.to_excel(writer)
# writer.save()

data_mat = df_2.values 

# Min-Max Scale 

scalers = {}
datas = [] 

df_scaled = pd.DataFrame(columns=cols)


############################################################
#  Fix below - I am scaling the whole data set together, when I should scale the train and test datasets separately
############################################################

for i in range(len(cols)): 
    scalers[cols[i]] = MinMaxScaler()
    #print('data', data_array[:,i])
    
    col_data = data_mat[:,i]
    col_data = np.reshape(col_data, (len(col_data), 1))
    
    #display(cols[i])
    #display(col_data)
    
    data = scalers[cols[i]].fit_transform( col_data )
    #print('scaled', data)
    data = np.reshape(data, (1, len(data)))
    df_scaled[cols[i]] = data[0]
    
df_scaled['Is Spike'] = df['Is Spike']
df_scaled.dropna(inplace=True)
display(df_scaled.head())
display(df_scaled.tail())

C:\Users\Shoya\Anaconda2\envs\tensorflow\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Timestamp,Close,Volume_(BTC),Volume_(Currency),Date(UTC),Bitcoin (Adj.Overlap),Close Price % Change,Close Price % Change (Abs),Is Spike,BitstampTrend,CoinbaseTrend,EthereumTrend,BubbleTrend,CryptocurrencyTrend,HashingTrend,BlockchainTrend
0,1420840800,289.13,34.042422,9854.490388,2015-01-09 22:00:00,53.0,NaN,NaN,0,15.0,42.0,41.0,82.0,15.0,44.0,62.0
1,1420844400,292.15,36.628012,10733.474260,2015-01-09 23:00:00,52.0,0.010445,0.010445,1,16.0,48.0,39.0,86.0,25.0,44.0,53.0
2,1420848000,293.89,0.158015,46.439149,2015-01-10 00:00:00,52.0,0.005956,0.005956,1,17.0,53.0,45.0,90.0,27.0,50.0,53.0
3,1420851600,289.10,30.985828,8960.344674,2015-01-10 01:00:00,56.0,-0.016299,0.016299,-1,19.0,59.0,69.0,90.0,29.0,45.0,67.0
4,1420855200,288.59,7.694195,2224.105007,2015-01-10 02:00:00,53.0,-0.001764,0.001764,0,21.0,55.0,62.0,87.0,27.0,51.0,70.0


C:\Users\Shoya\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Shoya\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Shoya\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,Volume_BTC,Bitcoin_Adj,Price_lagged,BitstampTrend,CoinbaseTrend,EthereumTrend,BubbleTrend,CryptocurrencyTrend,HashingTrend,BlockchainTrend
0,-5.445876,0.000000,0.005938,0.060625,0.099091,0.143101,0.045462,0.076961,0.127833,0.000000
1,5.278593,0.074108,-0.016433,0.111226,0.107246,0.427444,0.000000,0.071459,-0.105361,0.234401
2,-1.393064,-0.055060,-0.001766,0.100083,-0.070204,-0.106972,-0.033902,-0.071459,0.125163,0.043803
3,4.178019,0.306374,-0.006571,0.000000,0.018019,0.291706,0.055880,-0.076961,-0.125163,0.028171
4,-6.770292,-0.149532,0.005808,-0.100083,-0.074108,-0.547640,0.000000,-0.127833,0.144581,0.130053


,Volume_BTC,Bitcoin_Adj,Price_lagged,BitstampTrend,CoinbaseTrend,EthereumTrend,BubbleTrend,CryptocurrencyTrend,HashingTrend,BlockchainTrend,Is Spike
1,0.776791,0.471970,0.484557,0.526899,0.565339,0.584618,0.535584,0.527141,0.328003,0.620269,1.0
2,0.463316,0.439996,0.538331,0.524515,0.506901,0.478824,0.512201,0.501026,0.390657,0.543009,1.0
3,0.725079,0.529463,0.520715,0.503101,0.535955,0.557747,0.574125,0.500021,0.322621,0.536672,-1.0
4,0.210661,0.416611,0.566098,0.481687,0.505616,0.391588,0.535584,0.490725,0.395935,0.577971,0.0
5,0.594148,0.445509,0.568881,0.503101,0.571612,0.487224,0.535584,0.652792,0.278450,0.431076,-1.0


,Volume_BTC,Bitcoin_Adj,Price_lagged,BitstampTrend,CoinbaseTrend,EthereumTrend,BubbleTrend,CryptocurrencyTrend,HashingTrend,BlockchainTrend,Is Spike
30193,0.455905,0.449846,0.546519,0.498689,0.561916,0.514671,0.571703,0.525508,0.278450,0.589149,1.0
30194,0.308749,0.457405,0.549938,0.498596,0.544880,0.502343,0.549518,0.522206,0.324627,0.471125,-1.0
30195,0.632915,0.453625,0.563428,0.451915,0.504870,0.497657,0.514576,0.516713,0.356639,0.544110,1.0
30196,0.629822,0.434124,0.506657,0.401379,0.519397,0.497629,0.542658,0.508787,0.553393,0.568882,-1.0
30197,0.564642,0.445374,0.493535,0.493590,0.524578,0.490222,0.521362,0.541482,0.316794,0.557062,-1.0


In [5]:
from keras.utils.np_utils import to_categorical

# split and reshape data to feed into RNN

# X_timestamp = df_scaled['Timestamp'].values
X_volume = df_scaled['Volume_BTC'].values
X_trends = df_scaled['Bitcoin_Adj'].values
X_lagged_price = df_scaled['Price_lagged'].values

Y_is_spike = df_scaled['Is Spike'].values 

# train_size = int(len(X_volume) * 0.80)
# train_size = int(train_size/data_length) * data_length

test_size_index = int(len(X_volume)/data_length)*data_length

train_size = int(len(X_volume) * 0.80)
train_size = int(train_size/data_length) * data_length

# train_size = int(len(X_volume) * 0.90)
# train_size = int(train_size/data_length) * data_length

# val_size = int(train_size * 0.95)
# val_size = int(val_size/data_length) * data_length

#print(val_size, train_size, len(X_volume))

X_train_volume = []
X_test_volume = [] 
X_train_trends = []
X_test_trends = []
X_train_lagged_price = []
X_test_lagged_price = []
Y_train_is_spike = [] 
Y_test_is_spike = [] 

for i in range(train_size-data_length):
    vol_temp = []
    trends_temp = []
    price_temp = []
    for j in range(data_length):
        vol_temp.append(X_volume[i+j])
        trends_temp.append(X_trends[i+j])
        price_temp.append(X_lagged_price[i+j])
    X_train_volume.append(vol_temp)
    X_train_trends.append(trends_temp)
    X_train_lagged_price.append(price_temp)
    
    Y_train_is_spike.append(Y_is_spike[i+data_length])

for i in range(test_size_index-train_size-data_length):
    vol_temp = []
    trends_temp = [] 
    price_temp = [] 
    for j in range(data_length):
        vol_temp.append(X_volume[train_size+i+j])
        trends_temp.append(X_trends[train_size+i+j])
        price_temp.append(X_lagged_price[train_size+i+j])
    X_test_volume.append(vol_temp)
    X_test_trends.append(trends_temp)
    X_test_lagged_price.append(price_temp)
    
    Y_test_is_spike.append(Y_is_spike[train_size+i+data_length])
    
X_train_volume = np.array(X_train_volume)
X_test_volume =  np.array(X_test_volume)
X_train_trends = np.array(X_train_trends)
X_test_trends = np.array(X_test_trends)
X_train_lagged_price = np.array(X_train_lagged_price)
X_test_lagged_price = np.array(X_test_lagged_price)
Y_train_is_spike =  np.array(Y_train_is_spike)
Y_test_is_spike = np.array(Y_test_is_spike)
    
    
Y_train_is_spike_onehot = to_categorical(Y_train_is_spike, num_classes=3)
Y_test_is_spike_onehot = to_categorical(Y_test_is_spike,num_classes=3)
display(Y_train_is_spike)

X_train_volume = np.reshape(X_train_volume, (X_train_volume.shape[0],data_length,1) ) 
X_train_trends = np.reshape(X_train_trends, (X_train_trends.shape[0],data_length,1) ) 
X_train_lagged_price = np.reshape(X_train_lagged_price, (X_train_lagged_price.shape[0], data_length, 1))

X_test_volume = np.reshape(X_test_volume, (X_test_volume.shape[0],data_length,1) ) 
X_test_trends = np.reshape(X_test_trends, (X_test_trends.shape[0],data_length,1) )  
X_test_lagged_price = np.reshape(X_test_lagged_price, (X_test_lagged_price.shape[0],data_length,1))

# split training into train and val 


# val_size = int(len(X_train_volume) * 0.80)
# val_size = int(val_size/data_length) * data_length

# X_train_volume, X_val_volume = X_train_volume[:val_size], X_train_volume[-(train_size-val_size):]
# X_train_trends, X_val_trends = X_train_trends[:val_size], X_train_trends[-(train_size-val_size):]
# X_train_lagged_price, X_val_lagged_price = X_train_lagged_price[:val_size], X_train_lagged_price[-(train_size-val_size):]
# Y_train_is_spike_onehot, Y_val_is_spike_onehot = Y_train_is_spike_onehot[:val_size], Y_train_is_spike_onehot[-(train_size-val_size):]

array([ 0.,  0., -1., ..., -1.,  1., -1.])

In [15]:


#features = ['Volume_BTC', 'Bitcoin_Adj', 'Price_lagged']#featur 
features = ['Price_lagged']

rnn = build_model(features, data_length) 

tensorboard_callback = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

history = rnn.fit(
    [
        #X_train_timestamp,
       # X_train_volume,
        #X_train_trends,
        X_train_lagged_price
    ],
    [
        Y_train_is_spike_onehot
    ]
    ,
#     validation_data=(
#         [
#             #X_val_timestamp,
#             X_val_volume,
#             #X_val_trends,
#             X_val_lagged_price
#         ],
#         [
#             Y_val_is_spike_onehot
#         ]),
    epochs=100,
    batch_size=64,
    callbacks=[
      tensorboard_callback
    ],
    verbose=1
)

Epoch 1/100
24132/24132 [==============================] - 11s - loss: 1.0758 - categorical_accuracy: 0.4385    
Epoch 2/100
24132/24132 [==============================] - 10s - loss: 1.0752 - categorical_accuracy: 0.4390    
Epoch 3/100
24132/24132 [==============================] - 10s - loss: 1.0750 - categorical_accuracy: 0.4390    
Epoch 4/100
24132/24132 [==============================] - 10s - loss: 1.0752 - categorical_accuracy: 0.4390    
Epoch 5/100
24132/24132 [==============================] - 10s - loss: 1.0749 - categorical_accuracy: 0.4390    
Epoch 6/100
24132/24132 [==============================] - 10s - loss: 1.0751 - categorical_accuracy: 0.4390    
Epoch 7/100
24132/24132 [==============================] - 10s - loss: 1.0750 - categorical_accuracy: 0.4390    
Epoch 8/100
24132/24132 [==============================] - 10s - loss: 1.0748 - categorical_accuracy: 0.4390    
Epoch 9/100
24132/24132 [==============================] - 10s - loss: 1.0749 - categorical_accu

24132/24132 [==============================] - 11s - loss: 1.0364 - categorical_accuracy: 0.4766    
Epoch 73/100
24132/24132 [==============================] - 10s - loss: 1.0360 - categorical_accuracy: 0.4776    
Epoch 74/100
24132/24132 [==============================] - 11s - loss: 1.0357 - categorical_accuracy: 0.4760    
Epoch 75/100
24132/24132 [==============================] - 11s - loss: 1.0358 - categorical_accuracy: 0.4758    
Epoch 76/100
24132/24132 [==============================] - 11s - loss: 1.0355 - categorical_accuracy: 0.4769    
Epoch 77/100
24132/24132 [==============================] - 10s - loss: 1.0355 - categorical_accuracy: 0.4770    
Epoch 78/100
24132/24132 [==============================] - 10s - loss: 1.0347 - categorical_accuracy: 0.4799    
Epoch 79/100
24132/24132 [==============================] - 11s - loss: 1.0347 - categorical_accuracy: 0.4763    
Epoch 80/100
24132/24132 [==============================] - 10s - loss: 1.0339 - categorical_accuracy

In [ ]:
from matplotlib import pyplot as plt

# plot history
plt.plot(history.history['loss'], label='train')
#plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
score = rnn.evaluate(
    [
        #X_test_timestamp,
        #X_test_volume,
        #X_test_trends,
        X_test_lagged_price
    ],
    [
        Y_test_is_spike_onehot
    ])

print('\n')
print("Accuracy: %.2f%%" % (score[1]*100))

In [ ]:
yhat = rnn.predict( 
    [
        #X_test_timestamp,
        #X_test_volume,
        #X_test_trends,
        X_test_lagged_price
    ],
    verbose=0
)

display(yhat)

inverted_yhat = np.argmax(yhat,axis=1) #returns INDICES of max 
onehot_to_val_dict = {0: 0, 1: 1, 2:-1 }

inverted_yhat_arr = np.asarray(inverted_yhat)
predicted = [onehot_to_val_dict[i] for i in inverted_yhat_arr]


df_pred_output = pd.DataFrame(predicted, columns=['predicted'])
df_pred_output['actual'] = Y_test_is_spike
#df_pred_output['index_output'] = inverted_yhat
display(df_pred_output.head())

correct = (df_pred_output['actual'].values == df_pred_output['predicted'].values)
accuracy = correct.sum() / correct.size
display(accuracy)

In [ ]:
correct = (df_pred_output['actual'].values == df_pred_output['predicted'].values)
incorrect = (df_pred_output['actual'].values != df_pred_output['predicted'].values)

fig, ax = plt.subplots(figsize=(15,15))

ax.plot(df.loc[correct, "Close"], 'bs', label='correct', )
ax.plot(df.loc[incorrect, "Close"], 'r^', label='incorrect', )
plt.legend()
plt.show();